In [34]:
# set up
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
from sagemaker import get_execution_role

role = get_execution_role()

region = boto3.Session().region_name
prefix = 'sagemaker/xgboost'
bucket='sagemaker-gcas' # put your s3 bucket name here, and create s3 bucket
# customize to your bucket where you have stored the data
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)

In [35]:
# get xgboost algorithm
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, 'xgboost')

In [36]:
# ensure folders and data are correct pathing/types
common_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": bucket_path + "/" + prefix + "/xgboost"
    },
    "ResourceConfig": {
        "InstanceCount": 1,   
        "InstanceType": "ml.m4.10xlarge",
        "VolumeSizeInGB": 5
    },
    "HyperParameters": {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "silent":"0",
        "objective": "multi:softmax",
        "num_class": "10",
        "num_round": "10"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 86400
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": bucket_path + "/"+ prefix+ '/train.csv',
                    "S3DataDistributionType": "FullyReplicated" 
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None"
        }
    ]
}

In [37]:
# single machine job params
single_machine_job_name = 'GCAS' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Job name is:", single_machine_job_name)

single_machine_job_params = copy.deepcopy(common_training_params)
single_machine_job_params['TrainingJobName'] = single_machine_job_name
single_machine_job_params['OutputDataConfig']['S3OutputPath'] = bucket_path + "/xgboost-single"
single_machine_job_params['ResourceConfig']['InstanceCount'] = 1

Job name is: GCAS2019-05-01-22-24-15


In [38]:
# create training job
sm = boto3.Session(region_name=region).client('sagemaker')
sm.create_training_job(**single_machine_job_params)

{'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:096527816675:training-job/gcas2019-05-01-22-24-15',
 'ResponseMetadata': {'RequestId': '577a011e-4b54-427b-b6c3-c2c3235e56a6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '577a011e-4b54-427b-b6c3-c2c3235e56a6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '98',
   'date': 'Wed, 01 May 2019 22:24:16 GMT'},
  'RetryAttempts': 0}}

In [39]:
# set up host
model_name=single_machine_job_name + '-mod'
print(model_name)

info = sm.describe_training_job(TrainingJobName=single_machine_job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data
}

create_model_response = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

GCAS2019-05-01-22-24-15-mod
https://s3-us-east-1.amazonaws.com/sagemaker-gcas/xgboost-single/GCAS2019-05-01-22-24-15/output/model.tar.gz
arn:aws:sagemaker:us-east-1:096527816675:model/gcas2019-05-01-22-24-15-mod


In [40]:
# create endpoint config
from time import gmtime, strftime

endpoint_config_name = 'GCAS' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

GCAS2019-05-01-22-27-40
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:096527816675:endpoint-config/gcas2019-05-01-22-27-40


In [41]:
# create endpoint
import time

endpoint_name = 'GCAS' #+ strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Updating':
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)


GCAS
arn:aws:sagemaker:us-east-1:096527816675:endpoint/gcas
Status: Updating
Status: Updating
Status: Updating
Status: Updating
Status: Updating
Status: Updating
Status: Updating
Status: Updating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:096527816675:endpoint/gcas
Status: InService


At this point, the training endpoint is complete. Add more cells for model accuracy.

In [42]:
# remove endpoint if done using
#sm.delete_endpoint(EndpointName=endpoint_name)